<a href="https://colab.research.google.com/github/agrigoridou/Tokenization-Zipf-s-Law-N-gram-Models/blob/main/%CE%92_N_gram_Language_Models_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [82]:
!pip install nltk numpy

# Προετοιμασία των δεδομένων

Πρώτα, θα φορτώσουμε τα δεδομένα και θα προετοιμάσουμε τα αρχεία για την εκπαίδευση και αξιολόγηση:

In [83]:
# Εισαγωγή βιβλιοθηκών και δεδομένων
import nltk
from nltk.corpus import treebank
from collections import Counter
import math
import random
import re
import pandas as pd

# Φορτώνουμε τα δεδομένα από το treebank του NLTK
nltk.download('treebank')

# Φορτώνουμε τα πρώτα 150 αρχεία για εκπαίδευση και τα υπόλοιπα 49 για αξιολόγηση
train_files = treebank.fileids()[:150]  # Αρχεία για εκπαίδευση
test_files = treebank.fileids()[150:]   # Αρχεία για αξιολόγηση

# Λήψη των προτάσεων από τα αρχεία εκπαίδευσης και αξιολόγησης
train_sents = [sent for file in train_files for sent in treebank.sents(file)]
test_sents = [sent for file in test_files for sent in treebank.sents(file)]

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Package treebank is already up-to-date!


# 1: Υπολογισμός perplexity στα κείμενα αξιολόγησης

In [84]:
def calculate_perplexity(model, sents, n=2):
    """
    Υπολογίζει το perplexity για ένα μοντέλο bigram ή trigram.
    Το n ορίζει το μέγεθος του n-gram (2 για bigram, 3 για trigram).
    """
    log_prob = 0  # Συνολικό λογάριθμο πιθανοτήτων
    total_ngrams = 0  # Συνολικός αριθμός n-grams

    # Για κάθε πρόταση
    for sent in sents:
        if n == 2:
            ngrams_list = bigrams(sent)  # Δημιουργία bigrams
        elif n == 3:
            ngrams_list = trigrams(sent)  # Δημιουργία trigrams

        # Για κάθε n-gram στην πρόταση
        for ngram in ngrams_list:
            # Αντιστοιχίζουμε την πιθανότητα του n-gram στο μοντέλο (με προσέγγιση αν δεν υπάρχει το n-gram)
            prob = model.get(ngram, 1e-6)
            log_prob += math.log(prob)  # Υπολογισμός του λογάριθμου της πιθανότητας
            total_ngrams += 1  # Αύξηση του αριθμού των n-grams

    # Υπολογισμός του perplexity
    perplexity = math.exp(-log_prob / total_ngrams)
    return perplexity

## Υπολογισμός perplexity για τα μοντέλα (bigram και trigram)

Το perplexity μετράει πόσο καλά το μοντέλο προβλέπει την επόμενη λέξη ή σύμβολο σε μια ακολουθία δεδομένων. Όσο χαμηλότερο είναι το perplexity, τόσο καλύτερη είναι η ικανότητα του μοντέλου να προβλέπει σωστά τις λέξεις με βάση τα δεδομένα εκπαίδευσης.

In [85]:
import pandas as pd

perplexity_bigram_1 = calculate_perplexity(bigram_model_1, test_sents, n=2)
perplexity_bigram_2 = calculate_perplexity(bigram_model_2, test_sents, n=2)
perplexity_trigram_1 = calculate_perplexity(trigram_model_1, test_sents, n=3)
perplexity_trigram_2 = calculate_perplexity(trigram_model_2, test_sents, n=3)

# Δημιουργία πίνακα με τα αποτελέσματα
data = {
    "Model": ["Bigram k=1", "Bigram k=0.01", "Trigram k=1", "Trigram k=0.01"],
    "Perplexity": [perplexity_bigram_1, perplexity_bigram_2, perplexity_trigram_1, perplexity_trigram_2]
}

# Δημιουργία DataFrame από τα δεδομένα
df = pd.DataFrame(data)
df

,Model,Perplexity
0,Bigram k=1,48383.258496
1,Bigram k=0.01,20958.120746
2,Trigram k=1,538873.515246
3,Trigram k=0.01,320035.619599


Το Bigram με k=0.01 φαίνεται να είναι το πιο αποδοτικό μοντέλο για το δεδομένο σύνολο αξιολόγησης, με το μικρότερο perplexity, γεγονός που υποδεικνύει ότι μπορεί να προβλέπει καλύτερα τις ακολουθίες λέξεων από τα άλλα μοντέλα. Τα Trigrams με k=1 ή χωρίς smoothing φαίνεται να είναι πιο δύσκολα να γενικευθούν και συνεπώς έχουν μεγαλύτερο perplexity.

# 2: Μετατροπή όλων των κειμένων σε πεζά

In [86]:
def preprocess_lowercase(sents):
    """
    Μετατρέπει όλα τα tokens σε πεζά.
    """
    return [[token.lower() for token in sent] for sent in sents]

## Προετοιμασία πεζών δεδομένων

In [87]:
train_lower = preprocess_lowercase(train_sents)
test_lower = preprocess_lowercase(test_sents)

## Υπολογισμός perplexity για τα μοντέλα με πεζά γράμματα

In [88]:
import pandas as pd

# Υπολογισμός perplexity για τα lowercase μοντέλα (bigram και trigram)
perplexity_bigram_lower_1 = calculate_perplexity(bigram_model_1, test_lower, n=2)
perplexity_bigram_lower_2 = calculate_perplexity(bigram_model_2, test_lower, n=2)
perplexity_trigram_lower_1 = calculate_perplexity(trigram_model_1, test_lower, n=3)
perplexity_trigram_lower_2 = calculate_perplexity(trigram_model_2, test_lower, n=3)

# Δημιουργία πίνακα με τα αποτελέσματα
data_lower = {
    "Model": ["Bigram (Lowercase) k=1", "Bigram (Lowercase) k=0.01", "Trigram (Lowercase) k=1", "Trigram (Lowercase) k=0.01"],
    "Perplexity": [perplexity_bigram_lower_1, perplexity_bigram_lower_2, perplexity_trigram_lower_1, perplexity_trigram_lower_2]
}

# Δημιουργία DataFrame από τα δεδομένα
df_lower = pd.DataFrame(data_lower)

df_lower


,Model,Perplexity
0,Bigram (Lowercase) k=1,20016.401066
1,Bigram (Lowercase) k=0.01,6759.136604
2,Trigram (Lowercase) k=1,390205.901466
3,Trigram (Lowercase) k=0.01,179792.221192


Η μετατροπή σε πεζά γράμματα φαίνεται να βελτιώνει το perplexity για τα μοντέλα bigram, καθώς οι τιμές μειώνονται αισθητά. Για τα μοντέλα trigram, η μετατροπή σε πεζά έχει μικρότερη επίδραση, αλλά υπάρχει και πάλι μια βελτίωση, αν και όχι τόσο δραστική όσο στα bigrams. Αυτό δείχνει ότι για μοντέλα χαμηλότερου επιπέδου (όπως τα bigrams), η μετατροπή σε πεζά γράμματα έχει σημαντική θετική επίδραση στην απόδοση του μοντέλου.

#3: Αντικατάσταση ψηφίων με '#'

In [89]:
import re

def replace_digits_with_hash(sents):
    """
    Αντικαθιστά όλα τα ψηφία με το σύμβολο '#'.
    """
    return [[re.sub(r'\d', '#', token) for token in sent] for sent in sents]


## Προετοιμασία δεδομένων με αντικατάσταση ψηφίων

In [90]:
train_hash = replace_digits_with_hash(train_sents)
test_hash = replace_digits_with_hash(test_sents)

## Υπολογισμός perplexity για τα μοντέλα με αντικατάσταση ψηφίων

In [91]:
import pandas as pd

# Υπολογισμός perplexity για τα hash δεδομένα (bigram και trigram)
perplexity_bigram_hash_1 = calculate_perplexity(bigram_model_1, test_hash, n=2)
perplexity_bigram_hash_2 = calculate_perplexity(bigram_model_2, test_hash, n=2)
perplexity_trigram_hash_1 = calculate_perplexity(trigram_model_1, test_hash, n=3)
perplexity_trigram_hash_2 = calculate_perplexity(trigram_model_2, test_hash, n=3)

# Δημιουργία πίνακα με τα αποτελέσματα
data_hash = {
    "Model": ["Bigram (Hash) k=1", "Bigram (Hash) k=0.01", "Trigram (Hash) k=1", "Trigram (Hash) k=0.01"],
    "Perplexity": [perplexity_bigram_hash_1, perplexity_bigram_hash_2, perplexity_trigram_hash_1, perplexity_trigram_hash_2]
}

# Δημιουργία DataFrame από τα δεδομένα
df_hash = pd.DataFrame(data_hash)

df_hash


,Model,Perplexity
0,Bigram (Hash) k=1,79285.109869
1,Bigram (Hash) k=0.01,39430.240567
2,Trigram (Hash) k=1,649137.763097
3,Trigram (Hash) k=0.01,446736.027853


Η αντικατάσταση των ψηφίων με τον χαρακτήρα ‘#’ επιδεινώνει το perplexity για όλα τα μοντέλα, ανεξαρτήτως του τύπου του n-gram. Αυτή η επίδραση είναι πιο έντονη για τα μοντέλα trigrams, που βασίζονται σε πιο σύνθετες αλληλουχίες λέξεων. Η αφαίρεση των ψηφίων αφαιρεί πληροφορίες που μπορεί να είναι κρίσιμες για την κατανόηση και πρόβλεψη των λέξεων από το μοντέλο, οδηγώντας σε αύξηση της αβεβαιότητας (perplexity).

# 4: Δημιουργία νέων προτάσεων

In [95]:
def generate_sentence(model, n=2, max_len=20, start_word="<BOS>", end_word="<EOS>"):
    """
    Δημιουργεί μια νέα πρόταση βασισμένη στο μοντέλο n-grams.
    Επιστρέφει την πρόταση μέχρι το μέγιστο μήκος (max_len).
    """
    sentence = [start_word]  # Το σύμβολο της αρχής της πρότασης
    while len(sentence) < max_len:
        last_word = sentence[-1]  # Το τελευταίο λέξη στην πρόταση
        possible_next_words = {key[1] for key in model if key[0] == last_word}  # Πιθανές επόμενες λέξεις

        if not possible_next_words:  # Αν δεν υπάρχουν επόμενες λέξεις, σταματάμε
            break

        # Επιλέγουμε τυχαία την επόμενη λέξη βάσει πιθανοτήτων
        next_word = random.choices(list(possible_next_words), weights=[model.get((last_word, word), 1e-6) for word in possible_next_words])[0]
        sentence.append(next_word)

        if next_word == end_word:  # Αν φτάσουμε στο τέλος της πρότασης
            break

    return ' '.join(sentence)  # Επιστρέφουμε την πρόταση ως κείμενο

## Δημιουργία νέων προτάσεων

In [98]:
# Δημιουργία νέων προτάσεων από το μοντέλο Bigram με k=0.01
new_sentence_1 = generate_sentence(bigram_model_2, n=2, start_word="<BOS>", end_word="<EOS>")
new_sentence_2 = generate_sentence(bigram_model_2, n=2, start_word="<BOS>", end_word="<EOS>")
new_sentence_3 = generate_sentence(bigram_model_2, n=2, start_word="<BOS>", end_word="<EOS>")


# Δημιουργία πίνακα με τα αποτελέσματα
data_sentences = {
    "Model": ["Νέα Πρόταση 1:", "Νέα Πρόταση 2:", "Νέα Πρόταση 3:"],
    "Generated Sentence": [new_sentence_1, new_sentence_2, new_sentence_3]
}

# Δημιουργία DataFrame από τα δεδομένα
df_sentences = pd.DataFrame(data_sentences)

df_sentences


,Model,Generated Sentence
0,Νέα Πρόταση 1:,<BOS> an <UNK> because state department of its...
1,Νέα Πρόταση 2:,"<BOS> britain , not *-2 to global market with ..."
2,Νέα Πρόταση 3:,<BOS> the american medical schools and has <UN...
